In [1]:
import h5py
import numpy as np
import os
from bm3d import bm3d
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Lambda
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt
from bm3d import bm3d_rgb, BM3DProfile
import sys
module_dir = "/global/u2/k/kberard/SCGSR/Research/Diamond/stock_models/bm3d-4.0.3/bm3d-4.0.3/examples" 
sys.path.insert(0, module_dir)
from experiment_funcs import get_experiment_noise, get_psnr, get_cropped_psnr
from PIL import Image
import math
import torch
import torch.nn as nn
import torch.optim as optim
from thop import profile
from einops import rearrange 
from einops.layers.torch import Rearrange, Reduce
from timm.models.layers import trunc_normal_, DropPath
from keras.models import load_model
import keras


module_dir_scu = "/global/u2/k/kberard/SCGSR/Research/Diamond/stock_models/SCUNet" 
sys.path.insert(0, module_dir_scu)
from models.network_scunet import SCUNet as SCUNet
from torch.utils.data import Dataset
from utils import utils_image as util
import joblib
from sklearn.preprocessing import MinMaxScaler

2025-08-05 16:09:01.966332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754435341.980044 1644696 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754435341.984269 1644696 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754435341.996258 1644696 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754435341.996270 1644696 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754435341.996271 1644696 computation_placer.cc:177] computation placer alr

In [2]:
def run_denoising_pipeline(noisy_path, ref_path, model_name, save_path='denoised_output.npy'):
    # Load data
    with h5py.File(ref_path, 'r') as f:
        ref_d = f['density'][:]
    with h5py.File(noisy_path, 'r') as f:
        test_d = f['density'][:]

    model_dict = {
        'scunet_trained': scunet_trained_test_pipeline,
        'scunet_pre': scunet_pre_test_pipeline,
        'scunet_ft': scunet_FT_test_pipeline,
        'nature': Nature_test_pipeline,
        'CAE': CAE_test_pipeline,
        'CAE_3D': CAE_3D_test_pipeline,
        'bm3d': bm3d_test_pipeline
    }

    if model_name not in model_dict:
        raise ValueError(f"Unsupported model: {model_name}. Choose from: {list(model_dict.keys())}")

    # Capture printed metrics and denoised output
    import io
    from contextlib import redirect_stdout

    # Redirect stdout to capture
    f = io.StringIO()
    with redirect_stdout(f):
        model_dict[model_name](test_d, ref_d)
    print(f.getvalue())  # Print all output once

    # Save denoised result (retrieved from function scope if needed)
    if model_name == 'CAE_3D':
        denoised_volume = CAE_3D_mod.predict(test_d.reshape(1,64,64,64,1)).reshape(64,64,64)
    elif model_name in ['CAE', 'scunet_trained', 'scunet_pre', 'scunet_ft', 'nature', 'bm3d']:
        # All these use encode_rgb → decode_rgb
        denoised_rgb = encode_voxel_to_rgb(test_d)
        if model_name == 'CAE':
            denoised_rgb = CAE_mod.predict(denoised_rgb)
        elif model_name == 'nature':
            denoised_rgb = nature_mod.predict(denoised_rgb)
        elif model_name == 'scunet_trained':
            torch.serialization.add_safe_globals([SCUNet])
            model = torch.load('scunet_trained', weights_only=False)
            model.eval()
            denoised_rgb = np.zeros_like(denoised_rgb)
            for i in range(64):
                img_tensor = torch.from_numpy(np.transpose(denoised_rgb[i], (2, 0, 1))).float().unsqueeze(0).to('cpu')
                with torch.no_grad():
                    output_tensor = model(img_tensor)
                denoised_rgb[i] = np.transpose(output_tensor.squeeze().cpu().numpy(), (1, 2, 0))
        # All decoded to voxel grid
        denoised_volume = decode_rgb_to_voxel(denoised_rgb)
    else:
        raise NotImplementedError("Denoised output extraction not defined for this model")

    np.save(save_path, denoised_volume)
    print(f"Denoised volume saved to {save_path}")
    return(denoised_volume)


In [3]:
den = run_denoising_pipeline(
    noisy_path='/pscratch/sd/k/kberard/SCGSR/Data/diamond_1x1x1_bfd/density_data/vmc_J2/density_tot_vmc_mean_0000655360.h5',
    ref_path='/global/u2/k/kberard/SCGSR/Research/Diamond/Data/density_tot_ref.h5',
    model_name='CAE_3D',  # or 'bm3d', 'scunet_trained', etc.
    save_path='denoised_CAE_3D_output.npy'
)


NameError: name 'scunet_trained_test_pipeline' is not defined

In [ ]:
print("here")